In [2]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 47.8 MB/s 


In [3]:
import codecs
with codecs.open('data2.txt', encoding='utf-8') as f:
    data = [문장.split('\t') for 문장 in f.read().splitlines()]
    data = data[1:]

In [5]:
긍정문장_data = [문장[1] for 문장 in data if 문장[2] == '1']
부정문장_data = [문장[1] for 문장 in data if 문장[2] == '0']

In [8]:
len(긍정문장_data), len(부정문장_data)

(74827, 75173)

In [11]:
type(긍정문장_data)

list

데이터 전처리(단어 토큰 -> 형태소 토큰화)

In [15]:
from konlpy.tag import Okt
from tqdm import tqdm #작업의 진행상황을 알려주는 패키지
from nltk.util import ngrams
tg = Okt()
def tk_f(문장): #토큰화된 data는 튜플을 원소로 가지는 리스트 자료형이 된다. 이를 다시 문자열로 이루어진 데이터로 만들기 위한 함수
    tk_d = ['/'.join(i) for i in tg.pos(문장)]
    return tk_d

In [16]:
tg.pos(긍정문장_data[0])

[('흠', 'Noun'),
 ('...', 'Punctuation'),
 ('포스터', 'Noun'),
 ('보고', 'Noun'),
 ('초딩', 'Noun'),
 ('영화', 'Noun'),
 ('줄', 'Noun'),
 ('....', 'Punctuation'),
 ('오버', 'Noun'),
 ('연기', 'Noun'),
 ('조차', 'Josa'),
 ('가볍지', 'Adjective'),
 ('않구나', 'Verb')]

In [17]:
tk_f(긍정문장_data[0])

['흠/Noun',
 '.../Punctuation',
 '포스터/Noun',
 '보고/Noun',
 '초딩/Noun',
 '영화/Noun',
 '줄/Noun',
 '..../Punctuation',
 '오버/Noun',
 '연기/Noun',
 '조차/Josa',
 '가볍지/Adjective',
 '않구나/Verb']

In [18]:
end_l = []
for 문장 in tqdm(긍정문장_data):
    tk_data = tk_f(문장)
    bg = ngrams(tk_data, 2, pad_left=True, pad_right=True, left_pad_symbol='SS', right_pad_symbol = 'SE')
    end_l += [t for t in bg]

100%|██████████| 74827/74827 [04:53<00:00, 255.11it/s]


In [19]:
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
cfd= ConditionalFreqDist(end_l)
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [28]:
st = 'SS'
all_l = []
import random
#random.seed(2)
while True:
    st = cpd[st].generate()
    if st=='SE':
        break
    d = st.split('/')[0]
    all_l.append(d)
n_data = ' '.join(all_l)

n_data

'파란만장한 복수 를 찍은건 가요 결국 .. 가족 을 관찰 해 줬다 선택 보다 재미 는 어떻 게 너무 재밌었어요 !'

In [30]:
tg.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [33]:
def 한글_정리_생성(c='SS', sd=8, cpd=cpd):
    import random
    random.seed(sd)
    all_l=[]
    while True:
        w=cpd[c].generate()
        if w=='SE':
            break
        w2=w.split('/')[0]
        pos=w.split('/')[1]
        
        if c == 'SS':
            all_l.append(w2.title())
        
        elif c in ["'", "\"", '(']:
            all_l.append(w2)
        #뒷글자가 아래의 문자로 시작한다면 그냥 넣음
        elif w in ["'", "\'", ",", ".", ")", ":", ";", "?"]:
            all_l.append(w2)
        #조사, 구두점, 접미사 등의 단어는 바로 붙임
        elif pos in ['Josa', 'Punctuation','Suffix']:
            all_l.append(w2)
        #특정 문자 또한 바로 붙임
        elif w in ['임/Noun','것/Noun','는걸/Noun','릴때/Noun','되다/Verb']:
            all_l.append(w2)
        #그밖의 것들은 공백과 함께 넣어서 문장처럼 보이도록 한다.
        else:
            all_l.append(' '+w2)
        #리스트에 append한 후 현재의 뒷글자를 앞글자로 바꾸어주고 while문 다시 시작
        c=w
    return ''.join(all_l)


In [34]:
한글_정리_생성()

'영상 진짜 웃기 기도하고 이제야 알았다니 떡잎부터 사소한것하나까지 알고있는 진실을 훔쳐 보고 다시 한번쯤 보면 공감이 넘 좋아요 아.. 인생을 감수 하는 대통령의 코믹 할 영화'

In [ ]:
end_l_1 = []
for 문장 in tqdm(부정문장_data):
    tk_data = tk_f(문장)
    bg = ngrams(tk_data, 2, pad_left=True, pad_right=True, left_pad_symbol='SS', right_pad_symbol = 'SE')
    end_l_1 += [t for t in bg]

In [ ]:
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
cfd1= ConditionalFreqDist(end_l_1)
cpd1 = ConditionalProbDist(cfd1, MLEProbDist) 
#부정 문장 만든 것

In [ ]:
한글_정리_생성(cpd=cpd1)